# Język Python - Wykład 4.

## Context Manager 

Problem:
    
    set things up
    do something
    tear things down

Rozwiązanie 1.

    set things up
    try:
        do something
    finally:
        tear things down

Rozwiązanie 2.

    def controlled_execution(callback):
        set things up
        try:
            callback(thing)
        finally:
            tear things down

    def my_function(thing):
        do something

    controlled_execution(my_function)

Rozwiązanie 3.

    def controlled_execution():
        set things up
        try:
            yield thing
        finally:
            tear things down

    for thing in controlled_execution():
        do something with thing

Rozwiązanie 4.

    class controlled_execution:
        def __enter__(self):
            set things up
            return thing
        def __exit__(self, type, value, traceback):
            tear things down

    with controlled_execution() as thing:
         some code

    with expression [as (targets)]:
        code block with context of "targets"

* Możliwość utworzenia dynamicznego (w runtime) kontekstu dla wykonania sekcji kodu
* Przykłady użycia: 
   * czytanie z pliku - w kontekście otwarcia i zamknięcia pliku
   * zakładanie i zwalnianie blokad ("lock")
   * zmiana i odtworzenie globalnego stanu
* Obiekt context managera musi implementować protokół: ``__enter__()`` i ``__exit__()``
* [PEP 343](http://www.python.org/dev/peps/pep-0343/)

In [ ]:
with open('L4.ipynb') as notebook:
    print(notebook.read(50))

In [ ]:
notebook

Konstrukcja:

        with EXPR as VAR:
            BLOCK

tłumaczy się jako:

        mgr = (EXPR)
        exit = type(mgr).__exit__  # Not calling it yet
        value = type(mgr).__enter__(mgr)
        exc = True
        try:
            try:
                VAR = value  # Only if "as VAR" is present
                BLOCK
            except:
                # The exceptional case is handled here
                exc = False
                if not exit(mgr, *sys.exc_info()):
                    raise
                # The exception is swallowed if exit() returns true
        finally:
            # The normal and non-local-goto cases are handled here
            if exc:
                exit(mgr, None, None, None)

In [ ]:
class MyContextManager():
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager() as f:
    print("Hello", f)

In [ ]:
class MyContextManager():
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)
        #return True

        
with MyContextManager():
    raise Exception()

### Context manager z użyciem dekoratora

In [ ]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name,)
    yield
    print("</%s>" % name,)

with tag("h1"):
    print("foo",)

print()

with tag("div"):
    print("foo",)


Przykład z tymczasowym folderem (za http://stackoverflow.com/questions/3012488/what-is-the-python-with-statement-designed-for)

In [ ]:
from tempfile import mkdtemp
from shutil import rmtree


@contextmanager
def temporary_dir(*args, **kwds):
    name = mkdtemp(*args, **kwds)
    try:
        yield name
    finally:
        rmtree(name)


with temporary_dir(".temp") as dirname:
    print("doing sth with",dirname)

## Metody specjalne c.d.

### ``__repr__`` - the “official” string representation of an object

In [ ]:
class T:
    pass

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

In [ ]:
class T:
    def __repr__(self):
        return 'T class'

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

### ``__str__`` - the “informal” or nicely printable string representation of an object.

In [ ]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)

print(Car('Porsche', '911', 3600))
print(repr(Car('Porsche', '911', 3600)))
print(str(Car('Porsche', '911', 3600)))

W przypadku, gdy metoda __str__ nie jest zdefiniowana, wywoływana jest metoda __repr__

### ``__hash__`` - (Hash values are integers. They are used to quickly compare dictionary keys during a dictionary lookup)

In [ ]:
class T:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return str(self.x)+" "+str(self.y)
        
    def __eq__(self,other):
        print("comparing to" + str(other))
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash(self.x)

d = {T(1,x) : x for x in range(100)}
T(1,10) in d

### ``__bool__`` - Called to implement truth value testing and the built-in operation

In [ ]:
bool(None)

In [ ]:
bool(0)

In [ ]:
bool([1,3,4])

In [ ]:
bool("")

In [ ]:
class T:
    def __len__(self):
        return 2
print(bool(T()))

In [ ]:
class T:
    def __bool__(self):
        print(id(self))
        if id(self) // 10 % 2 :
            return True
        else:
            return False
print(bool(T()))        

### ``__class__`` - klasa, do której należy instancja

In [ ]:
x = 5
x.__class__

In [ ]:
type(x)

In [ ]:
type(int)

### Emulacja typu kontenera (cd..)

In [ ]:
class T():
   
    c = [1, 2, 3]

    def __len__(self):
        return len(self.c)
    
    def __contains__(self, x):
        return x in self.c
    
    def __iter__(self):
        for x in self.c:
            yield x
    
t = T()
print(len(t))
print(2 in t)

In [ ]:
iter(t)

In [ ]:
list(iter(t))

## Porównanie

In [ ]:
# czy jest to ta sama wartość?
a = 100000
b = 100000
a == b

In [ ]:
# czy jest to ten sam obiekt?
a = 100000
b = 100000
a is b

In [ ]:
# pułapki optymalizacji CPythona
a = 1
b = 1
a==b, a is b

In [ ]:
# porównanie w przypadku klas
class T:
    pass
a = T()
b = T()
c = a
a==b,a==c

In [ ]:
# porównanie w przypadku klas
class T:
    def __eq__(self,other):
        return True
a = T()
b = T()
a==b

Atrybuty

In [ ]:
class Door:
    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [ ]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')
print(id(door1),id(door2))
print(id(door1.__class__),id(door2.__class__))
'0xb67e144c'

In [ ]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [ ]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')

In [ ]:
print(id(Door.colour),id(door1.colour),id(door2.colour))

In [ ]:
Door.colour, door1.colour, door2.colour

In [ ]:
Door.__dict__

In [ ]:
door1.__dict__

In [ ]:
door1.colour = "white"

In [ ]:
Door.open

In [ ]:
door1.open

In [ ]:
door1.__class__.__dict__['open'].__get__(door1)

## Dziedziczenie i MRO

In [ ]:
class SecurityDoor(Door):
    pass

In [ ]:
sdoor = SecurityDoor(1, 'closed')

In [ ]:
sdoor.__dict__

In [ ]:
SecurityDoor.__dict__

In [ ]:
SecurityDoor.__bases__

In [ ]:
sdoor.open

In [ ]:
sdoor.__class__.__dict__['open'].__get__(sdoor)

In [ ]:
sdoor.__class__.__bases__[0].__dict__['open'].__get__(sdoor)

In [ ]:
class A:
    def foo(self):
        print("A")
    def moo(self):
        pass
class B1(A):
    def foo(self):
        print("B1")
    def moo(self):
        pass
class B2(A):
    def foo(self):
        print("B2")
    def moo(self):
        pass
class C(B1,B2):
#    def foo(self):
#        print("C")
     pass
oc = C()
oc.foo()
print(C.__mro__)

### MRO

In [ ]:
class X: pass
class Y: pass
class A(X,Y): pass
class B(Y,X): pass
class C(A,B): pass
C.__mro__

# https://en.wikipedia.org/wiki/C3_linearization

In [ ]:
class F(): pass
class E(): pass
class D(): pass
class C(D,F): pass
class B(D,E): pass
class A(B,C): pass
print(A.__mro__) 

## Wyjątki też są klasami!

In [ ]:
5 + '5'

In [ ]:
try:
    5 + '5'
except TypeError:
    print('no-no')

In [ ]:
try:
    5 + '5'
except TypeError as e:
    print(type(e), e)

In [ ]:
try:
    5 + '5'
except (TypeError, KeyError) as e:
    print(type(e), e)

In [ ]:
try:
    dict()[3]
except TypeError as e:
    print(type(e), e)
except KeyError as e:
    print("it's only key error")

In [ ]:
class WrongAnswerError(Exception):
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!')
except WrongAnswerError:
    print('good good good')
finally:
    print('bye!')

In [ ]:
class A:
    pass

raise A

In [ ]:
BaseException.__bases__

In [ ]:
Exception.__bases__

In [ ]:
TypeError.__bases__

In [ ]:
KeyboardInterrupt.__bases__